# Lab 5. Data Sci. kNN Decision Trees

Name:
    
Student ID:
    
Email:

## Part 1: Pandas

Answer questions about data on flights to the United States.

Data: 2008.csv.bz2 in archive (note that it is not necessary to unpack this file — the pandas.read_csv function is able to read from archives automatically)

Code book of dataset: http://stat-computing.org/dataexpo/2009/the-data.html

* Which of the reasons for flight cancellation (CancellationCode) was the most frequent? (decoding codes can be found in the data description)

* Find the average, minimum and maximum distance traveled by plane.

* Does the minimum distance traveled look suspicious? On which days and on which flights was it? What distance was covered by the same flights on other days?

* What airport was the most departures from? What city is it located in?

* Find for each airport the average flight time (AirTime) for all flights departing from it. Which airport has the highest value of this indicator?

* Find the airport with the largest share of delayed (DepDelay> 0) flights. At the same time, exclude from consideration airports from which less than 1000 flights were sent (use the filter function after groupby).

In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("2008.csv.bz2")
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7.009728e+06,6.855029e+06,...,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,2.224200e+03,1.273224e+02,...,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,1.961716e+03,7.018731e+01,...,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,...,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.220000e+02,7.700000e+01,...,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,1.571000e+03,1.100000e+02,...,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,3.518000e+03,1.570000e+02,...,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01
max,2008.0,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.743000e+03,1.379000e+03,...,4.962000e+03,3.080000e+02,4.290000e+02,1.000000e+00,1.000000e+00,2.436000e+03,1.352000e+03,1.357000e+03,3.920000e+02,1.316000e+03


## Part 2: metric methods and categorical features

All further experiments are proposed to be conducted on the Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

In this task, it is proposed to predict whether an employee’s request for access to a particular resource will be approved. All signs are categorical.

File: amazon.csv in archive

Immediately read the data and create a partition for learning and control:

In [5]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [6]:
data.shape

(32769, 10)

In [7]:
# share of positive examples
data.ACTION.mean()

0.9421099209618847

In [9]:
# number of values of features
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

C:\dev\python\Anaconda35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1. Implement three distance functions on categorical features.

The easiest way is to define the metrics as [user-defined distance] (http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), then use the kNN implementation from sklearn (in this case use function predict_proba).

Try to pay special attention to the effectiveness of the code - when implementing metrics "in the forehead" you may encounter a very long execution time.

#### Count for each of the metrics the quality on the test sample `X_test` with the number of neighbors k = 10. A measure of quality - AUC-ROC.

### What is the best distance function?

#### 2 (bonus). Pick the best (on the test sample) the number of neighbors k for each of the distance functions. What is the best quality possible to get?

For the selection, you can use any means of sklearn.

#### 3. Implement the counters (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx) that will replace the categorical features on real.

Namely, each categorical attribute must be replaced by three:
1. The number of `counts` of objects in the training set with the same characteristic value.
2. The number of `successes` objects of the first class y = 1 in the training sample with the same attribute value.
3. The smoothed ratio of the two previous values: (`successes` + 1) / (` counts` + 2).

Since attributes containing information about the target variable may lead to retraining, it may be useful to do * folding *: split the training set into n parts, and for the i-th part, consider `counts` and` successes` for all the rest parts. For the test sample, counters are counted for the entire training sample. Implement this option. You can use n = 3.

#### On the AUC-ROC method, test the k closest neighbors with a Euclidean metric for a sample where categorical features are replaced by counters. Compare the AUC-ROC two options for forming the sample - with and without folding. Do not forget to choose the best number of neighbors k.

#### 4. Add paired signs to the original sample — that is, for each pair f(i), f(j),  i <j  of the original categorical signs, add a new categorical sign f(ij), the value of which is a concatenation of values f(i) and f(j) (preferably through some special character to avoid collisions). Count the counters for this sample, find the quality of the k nearest neighbors method with the best k (with and without folding).

## Part 3: Decisive trees and forests

#### 1. Take from the previous section a dataset with paired features, converted using counters without folding. Set up the decision tree by selecting the optimal values for the `max_depth` and` min_samples_leaf` parameters. What is the best AUC-ROC ?

#### 2. Set up a random forest, picking up a number of trees `n_estimators`, in which the error goes on the asymptote. What quality on the test sample does it give?